In [1]:
import sqlite3

# Connect to (or create) the database
conn = sqlite3.connect("employees.db")

# Open and read the SQL script
with open("employee_data.sql", "r") as file:
    sql_script = file.read()

# Execute the entire SQL script
cursor = conn.cursor()
cursor.executescript(sql_script)

# Commit changes and close the connection
conn.commit()
conn.close()

print("SQL script executed successfully.")

SQL script executed successfully.


In [15]:
conn = sqlite3.connect("employees.db")
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Preview data (replace `your_table_name` with the actual name)
cursor.execute("SELECT salary FROM employees WHERE name = 'Alice';")
rows = cursor.fetchall()
print("Sample data:", rows)

conn.close()

Tables: [('employees',)]
Sample data: [(62000,)]


In [19]:
from langchain.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
import sqlite3

# Connect to SQLite database
def query_employee_db(sql_query: str) -> str:
    try:
        conn = sqlite3.connect("employees.db")
        cursor = conn.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        conn.close()
        return str(results)
    except Exception as e:
        return f"Database error: {e}"

# Define the database tool
@tool
def employee_db_tool(query: str) -> str:
    """
    Useful for answering questions about employees from the company database.
    The database has a table called 'employees' with following columns:
    - id INTEGER PRIMARY KEY,
    - name TEXT,
    - department TEXT,
    - salary INTEGER
    Input should be a full SQL query string (e.g., "SELECT salary FROM employees WHERE department = 'Marketing'";)
    """
    return query_employee_db(query)

llm = Ollama(model="gemma3:4b")

# Initialize agent
tools = [employee_db_tool]

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run query
# question = "how much is the earning of Alice?"
question = "can you please tell me which section Alice works in?"
# response = agent.run(question)
response = agent.invoke({"input": question})

print("\nResponse:", response)



> Entering new AgentExecutor chain...
I need to find out Alice's department. I can use the employee_db_tool to query the database for Alice's information.
Action: employee_db_tool
Action Input: "SELECT department FROM employees WHERE name = 'Alice'"
Observation: [('Marketing',)]
Thought:Alice works in the Marketing department.
Final Answer: Marketing

> Finished chain.

Response: {'input': 'can you please tell me which section Alice works in?', 'output': 'Marketing'}
